In [ ]:
from __future__ import annotations
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import numpy as np

# Load data and examples

In [ ]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [ ]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples

# Part A

In [ ]:
todays_examples[0] = todays_examples[0]._replace(answer_a="10092")

In [ ]:
def move_box(box_bool_map: np.ndarray, i, j, direction: str, box_to_place=False):
    nx, ny = box_bool_map.shape
    if i < 0 or i >= nx or j < 0 or j >= ny or box_bool_map[i][j] == 2:
        return False

    # If this space is not occupy it, place either a box,
    # or the robot (bool_map stays false)
    if box_bool_map[i][j] == False:
        box_bool_map[i][j] = box_to_place
        return True

    # If this space is occupied, try to move the box in the direction
    place_to_move = False
    if direction == "<" and j > 0:
        place_to_move = move_box(box_bool_map, i, j - 1, direction, True)
    if direction == ">" and j < ny - 1:
        place_to_move = move_box(box_bool_map, i, j + 1, direction, True)
    if direction == "^" and i > 0:
        place_to_move = move_box(box_bool_map, i - 1, j, direction, True)
    if direction == "v" and i < nx - 1:
        place_to_move = move_box(box_bool_map, i + 1, j, direction, True)

    return place_to_move

In [ ]:
def part_a(data: str) -> str:
    map_data, moves_data = data.split("\n\n")
    moves_data = moves_data.replace("\n", "")

    char_map = np.array([[char for char in line] for line in map_data.splitlines()])
    robot_position = np.argwhere(char_map == "@")[0]
    box_bool_map = ((char_map == "O") | (char_map == "#")).astype(int)
    box_bool_map[char_map == "#"] = 2

    for move in moves_data:
        potential_new_position = robot_position.copy()
        if move == "<":
            potential_new_position[1] -= 1
        if move == ">":
            potential_new_position[1] += 1
        if move == "^":
            potential_new_position[0] -= 1
        if move == "v":
            potential_new_position[0] += 1

        move_allowed = move_box(
            box_bool_map, potential_new_position[0], potential_new_position[1], move
        )
        if move_allowed:
            robot_position = potential_new_position
            box_bool_map[tuple(robot_position)] = 0

    result = 0
    for d_top, d_left in np.argwhere(box_bool_map == 1):
        result += d_top * 100 + d_left

    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [ ]:
todays_examples[0] = todays_examples[0]._replace(answer_b="9021")

In [ ]:
np.set_printoptions(threshold=10_000)

In [ ]:
def double_map_data(map_data: str) -> str:
    map_data = re.sub("#", "##", map_data)
    map_data = re.sub("O", "[]", map_data)
    map_data = re.sub(r"\.", "..", map_data)
    map_data = re.sub("@", "@.", map_data)
    return map_data

In [ ]:
def is_move_possible(box_map: np.ndarray, i, j, direction: str):
    nx, ny = box_map.shape
    if i < 0 or i >= nx or j < 0 or j >= ny or box_map[i][j] == "#":
        return False

    # If this space is not occupy it we can move
    if box_map[i][j] == ".":
        return True

    # If this space is occupied, try to move the boxes in the given direction
    rearrangment_possible = False
    if direction == "<" and j > 0:
        rearrangment_possible = is_move_possible(box_map, i, j - 1, direction)
    if direction == ">" and j < ny - 1:
        rearrangment_possible = is_move_possible(box_map, i, j + 1, direction)
    if direction == "^" and i > 0:
        rearrangment_possible = is_move_possible(box_map, i - 1, j, direction)
        if box_map[i][j] == "[":
            rearrangment_possible &= is_move_possible(box_map, i - 1, j + 1, direction)
        if box_map[i][j] == "]":
            rearrangment_possible &= is_move_possible(box_map, i - 1, j - 1, direction)
    if direction == "v" and i < nx - 1:
        rearrangment_possible = is_move_possible(box_map, i + 1, j, direction)
        if box_map[i][j] == "[":
            rearrangment_possible &= is_move_possible(box_map, i + 1, j + 1, direction)
        if box_map[i][j] == "]":
            rearrangment_possible &= is_move_possible(box_map, i + 1, j - 1, direction)

    return rearrangment_possible

In [ ]:
def move_to_position(
    box_map: np.ndarray, i, j, direction: str, field_to_place: str
) -> None:
    nx, ny = box_map.shape
    current_field_value = box_map[i][j]
    box_map[i][j] = field_to_place

    # if field empty, we just occupied it. The function is done
    if current_field_value == ".":
        return

    if direction == "<" and j > 0:
        move_to_position(box_map, i, j - 1, direction, current_field_value)
    if direction == ">" and j < ny - 1:
        move_to_position(box_map, i, j + 1, direction, current_field_value)
    if direction == "^" and i > 0:
        move_to_position(box_map, i - 1, j, direction, current_field_value)
        if current_field_value == "[":
            move_to_position(box_map, i - 1, j + 1, direction, "]")
            box_map[i][j + 1] = "."
        if current_field_value == "]":
            move_to_position(box_map, i - 1, j - 1, direction, "[")
            box_map[i][j - 1] = "."
    if direction == "v" and i < nx - 1:
        move_to_position(box_map, i + 1, j, direction, current_field_value)
        if current_field_value == "[":
            move_to_position(box_map, i + 1, j + 1, direction, "]")
            box_map[i][j + 1] = "."
        if current_field_value == "]":
            move_to_position(box_map, i + 1, j - 1, direction, "[")
            box_map[i][j - 1] = "."

In [ ]:
def check_and_execute_move(
    char_map: np.ndarray, robot_position: np.ndarray, move: str
) -> None:
    potential_new_position = robot_position.copy()
    if move == "<":
        potential_new_position[1] -= 1
    if move == ">":
        potential_new_position[1] += 1
    if move == "^":
        potential_new_position[0] -= 1
    if move == "v":
        potential_new_position[0] += 1

    if is_move_possible(
        char_map, potential_new_position[0], potential_new_position[1], move
    ):
        move_to_position(
            char_map, potential_new_position[0], potential_new_position[1], move, "@"
        )
        char_map[tuple(robot_position)] = "."
        robot_position[:] = potential_new_position

In [ ]:
def part_b(data: str) -> str:
    map_data, moves_data = data.split("\n\n")
    moves_data = moves_data.replace("\n", "")

    map_data = double_map_data(map_data)

    char_map = np.array([[char for char in line] for line in map_data.splitlines()])
    robot_position = np.argwhere(char_map == "@")[0]
    for move in moves_data:
        check_and_execute_move(char_map, robot_position, move)

    result = 0
    for d_top, d_left in np.argwhere((char_map == "[") | (char_map == "O")):
        result += d_top * 100 + d_left

    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_b != "":
        print(
            f"Example {example_index} part b: {part_b(example.input_data)} (expected {example.answer_b})"
        )
        assert part_b(str(example.input_data)) == example.answer_b
submit(part_b(data), part="b", year=puzzle_year, day=puzzle_day)